In [1]:
import sys
print(sys.version)

3.12.4 (v3.12.4:8e8a4baf65, Jun  6 2024, 17:33:18) [Clang 13.0.0 (clang-1300.0.29.30)]


In [1]:
import spacy
import dateparser
import pandas as pd
from datetime import datetime, timedelta
from nltk.corpus import wordnet as wn
import nltk


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# Download WordNet if not already downloaded
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/snehaagarwal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
# Seed words (we fetch synonyms dynamically)
SEED_HIGH_PRIORITY = ["urgent", "important", "critical", "necessary", "essential", "immediately"]
SEED_MEDIUM_PRIORITY = ["quick", "soon", "this week", "next few days", "moderate", "prompt"]
SEED_LOW_PRIORITY = ["not a priority", "low preference", "whenever", "someday", "optional"]


In [5]:
def get_synonyms(seed_words):
    """Fetches synonyms for the given words using WordNet (NLTK)."""
    synonyms = set()
    for word in seed_words:
        for syn in wn.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name().replace("_", " "))  # Convert underscores to spaces
    return synonyms

In [6]:
# Dynamically fetched synonyms
HIGH_PRIORITY_WORDS = get_synonyms(SEED_HIGH_PRIORITY)
MEDIUM_PRIORITY_WORDS = get_synonyms(SEED_MEDIUM_PRIORITY)
LOW_PRIORITY_WORDS = get_synonyms(SEED_LOW_PRIORITY)

In [7]:
def extract_deadline(task_description):
    """Extracts deadline from task description using NLP & dateparser"""
    doc = nlp(task_description)
    
    for ent in doc.ents:
        if ent.label_ == "DATE":
            return dateparser.parse(ent.text)  # Convert natural text to a date

    return None  # No deadline found


In [8]:
def lemmatize_task(task_description):
    """Converts task words into their base form using spaCy Lemmatization"""
    doc = nlp(task_description.lower())  # Convert to lowercase and process
    lemmatized_words = {token.lemma_ for token in doc}  # Extract unique lemmas
    return lemmatized_words

In [9]:
def determine_priority_based_on_keywords(task_description):
    """Assigns priority based on lemmatized urgency words if no deadline is mentioned"""
    lemmatized_words = lemmatize_task(task_description)

    if HIGH_PRIORITY_WORDS & lemmatized_words:  # If any high-priority words are found
        return "High"
    elif MEDIUM_PRIORITY_WORDS & lemmatized_words:
        return "Medium"
    elif LOW_PRIORITY_WORDS & lemmatized_words:
        return "Low"
    else:
        return "Low"  # Default if no clear priority words are found

In [10]:
def assign_priority(task_description):
    """Determines priority based on deadline (if available) or lemmatized keywords"""
    deadline = extract_deadline(task_description)

    if deadline:  # If a deadline exists, calculate priority based on days left
        days_left = (deadline - datetime.now()).days

        if days_left <= 0:
            return "High"
        elif days_left <= 2:
            return "High"
        elif days_left <= 5:
            return "Medium"
        else:
            return "Low"
    else:
        return determine_priority_based_on_keywords(task_description)  # Use dynamically fetched priority words

In [14]:
# Example tasks
tasks = [
    "Submit project by March 5",  # Deadline case
    "Fix critical server issue",  # "Critical" → Should be High
    "This task is essential",  # "Essential" → Should be High
    "Quickly done this work on time.",  # "Quickly" → Should be Medium
    "Take a vacation whenever",  # "Whenever" → Should be Low
    "Prepare presentation for next Monday",
    "Plan vacation next month",
    "Schedule a meeting sometime",  # No priority keyword → Default Low
    "Handle an indispensable update",  # "Indispensable" → Should be High (synonym of Important)
    "Fix an imperative bug immediately!",  # "Imperative" → High (synonym of Critical)
    "Complete this optional review",  # "Optional" → Should be Low
]

In [15]:
# Process tasks
task_data = []
for task in tasks:
    priority = assign_priority(task)

    task_data.append({
        "Task": task,
        "Priority": priority
    })


In [16]:
# Convert to DataFrame (for frontend display)
df = pd.DataFrame(task_data)
print(df)

                                    Task Priority
0              Submit project by March 5     High
1              Fix critical server issue     High
2                 This task is essential     High
3        Quickly done this work on time.   Medium
4               Take a vacation whenever      Low
5   Prepare presentation for next Monday      Low
6               Plan vacation next month      Low
7            Schedule a meeting sometime      Low
8         Handle an indispensable update     High
9     Fix an imperative bug immediately!     High
10         Complete this optional review      Low
